In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Convert panoramic videos to perspective videos

#### Make folder

In [ ]:
import os
def mkdir_ifnotexists(dir):
    if os.path.exists(dir):
        return
    os.mkdir(dir)

## Convert panoramic view to perspective view

###Frame processing

####Split up-down stereo image into left eye and right eye

In [ ]:
import cv2 as cv 
import numpy as np 

def image_split(img):
    left = img[0:int(img.shape[0]/2)]
    right = img[int(img.shape[0]/2):int(img.shape[0])]
    return left, right

###Convert panoramic image to perspective image based on view position

In [ ]:
import os
import sys
import cv2
import numpy as np

class Equirec2Perspec:
    # _init_ function
    # Input: 1. img: current frame of the panoramic video
    def __init__(self, img):
        self._img = img
        [self._height, self._width, _] = self._img.shape
    
    # Function convert equirectangular image to perspective
    # view based on the view position.
    # Input: 
    # 1. wFOV: horizontal field of view in degrees
    # 2. THETA: left/right angle in degrees of view center(right direction is positive, left direction is negative)
    # 3. PHI: up/down angle in degrees of view center(up direction is positive, down direction is negative)
    # 4. height, width: height/width of the output viewport image, should fit the resolution of each eye's viewport
    def GetPerspective(self, FOV, THETA, PHI, height, width):
        # set radius for the sphere which 
        # the equirectangular image is wrapped to
        RADIUS = 1
        
        # height, width of the input frame
        equ_h = self._height
        equ_w = self._width

        # center of the input frame
        equ_cx = (equ_w - 1) / 2.0
        equ_cy = (equ_h - 1) / 2.0

        # set vertical field of view based 
        # on output image size and horizontal 
        # field of view
        wFOV = FOV
        hFOV = float(height) / width * wFOV
        
        # center of the output image
        c_x = (width - 1) / 2.0
        c_y = (height - 1) / 2.0

        # horizontal length of view:
        # w_len = 2 * radius * tan(wFOV/2)
        w_len = 2 * RADIUS * np.tan(np.radians(wFOV / 2.0))
        # each pixels from frame represents 
        # how many units of the horizontal length of view
        w_interval = w_len / (width - 1)

        # same precedure for viertical view
        h_len = 2 * RADIUS * np.tan(np.radians(hFOV / 2.0)) 
        h_interval = h_len / (height - 1)
        
        # x_map: radius, distance between the viewport to the sphere center
        # y_map: horizontal distance between each image pixel and image center
        # z_map: vertical distance between each image pixel and image center
        x_map = np.zeros([height, width], np.float32) + RADIUS
        y_map = np.tile((np.arange(0, width) - c_x) * w_interval, [height, 1])
        z_map = -np.tile((np.arange(0, height) - c_y) * h_interval, [width, 1]).T
        # distance between the sphere center and each pixel at viewport image\
        # D = sqrt(radius^2 + horizontal_distance^2 + vertical_distance^2)
        D = np.sqrt(x_map**2 + y_map**2 + z_map**2)
        xyz = np.zeros([height, width, 3], np.float)
        # normalize to the sphere that equirectangular image is wrapped to
        xyz[:, :, 0] = (RADIUS / D * x_map)[:, :]
        xyz[:, :, 1] = (RADIUS / D * y_map)[:, :]
        xyz[:, :, 2] = (RADIUS / D * z_map)[:, :]
        
        # unit vector along vertical rotation axis
        vertical_axis = np.array([0.0, 1.0, 0.0], np.float32)
        # unit vector along horizontal rotation axis
        horizontal_axis = np.array([0.0, 0.0, 1.0], np.float32)
        # Rodrigues' rotation formula
        [R1, _] = cv2.Rodrigues(horizontal_axis * np.radians(THETA))
        [R2, _] = cv2.Rodrigues(np.dot(R1, vertical_axis) * np.radians(-PHI))

        # rotate the viewport
        xyz = xyz.reshape([height * width, 3]).T
        xyz = np.dot(R1, xyz)
        xyz = np.dot(R2, xyz).T
        # convert distance to latitude and longitude
        lat = np.arcsin(xyz[:, 2] / RADIUS)
        lon = np.zeros([height * width], np.float)
        theta = np.arctan(xyz[:, 1] / xyz[:, 0])
        
        # mask to crop out the subimages from equirectangular image
        idx1 = xyz[:, 0] > 0
        idx2 = xyz[:, 1] > 0

        idx3 = ((1 - idx1) * idx2).astype(np.bool)
        idx4 = ((1 - idx1) * (1 - idx2)).astype(np.bool)
        
        lon[idx1] = theta[idx1]
        lon[idx3] = theta[idx3] + np.pi
        lon[idx4] = theta[idx4] - np.pi

        # cooridinates of the mask at equirectangular image, in pixels
        lon = lon.reshape([height, width]) / np.pi * 180
        lat = -lat.reshape([height, width]) / np.pi * 180
        lon = lon / 180 * equ_cx + equ_cx
        lat = lat / 90 * equ_cy + equ_cy
        
        # sample equirectangular image based on coordinates
        persp = cv2.remap(self._img, lon.astype(np.float32), lat.astype(np.float32), cv2.INTER_CUBIC, borderMode=cv2.BORDER_WRAP)
        return lon.astype(np.int), lat.astype(np.int),persp

In [ ]:
import glob
import numpy as np
from pathlib import Path
import cv2 as cv
import csv
from numpy import save

def perspectiveSotre(lon, lat, videoName):
  frame_pth='./frames/'
  mkdir_ifnotexists(frame_pth)
  path = './videos/'+videoName+'.mp4'
  # DEPTH_VISUALIZATION_SCALE = 2048
  # The video feed is read in as 
  # a VideoCapture object 
  cap = cv.VideoCapture(str(path)) 
  
  # ret = a boolean return value from 
  # getting the frame, first_frame = the 
  # first frame in the entire video sequence 
  ret, first_frame = cap.read() 


  first_left = image_split(first_frame)[0]
  first_left_pers = Equirec2Perspec(first_left)
  [_, _, first_left_pers] = first_left_pers.GetPerspective(88, lon, lat, 1440, 1600)
  cv.imwrite(frame_pth+'frame_'+
              '{0:0=6d}'.format(int(cap.get(cv.CAP_PROP_POS_FRAMES)))+'.png', first_left_pers)


  # Creates an image filled with zero 
  # intensities with the same dimensions  
  # as the frame 
  mask = np.zeros_like(first_left_pers) 
      
  # Sets image saturation to maximum 
  mask[..., 1] = 255        


  n = 0

  while(cap.isOpened()): 
    print(cap.get(cv.CAP_PROP_POS_FRAMES))
    ret, frame = cap.read() 
    if ret == False:
      break
        
    # Image per eye
    left_img = image_split(frame)[0]
    left_img = Equirec2Perspec(left_img)
    [_, _, left_img] = left_img.GetPerspective(107, lon, lat, 1440, 1600)
    frame = left_img
    cv.imwrite(frame_pth+'frame_'+
              '{0:0=6d}'.format(int(cap.get(cv.CAP_PROP_POS_FRAMES)))+'.png', left_img)

    n = n +1
    
  # The following frees up resources and 
  # closes all windows 
  cap.release()
  cv.destroyAllWindows() 


# Setup and Install FlowNet2


## Download compatible Torch

In [ ]:
!pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu90/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu90/torch_stable.html
     |████████████████████████████████| 591.8MB 27kB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.0.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.19.9 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.19.9 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


## Download and setup FlowNet2 files

In [ ]:
import os
# get flownet2-pytorch source
#!git clone https://github.com/Gauravv97/flownet2-pytorch.git
!git clone https://github.com/NVIDIA/flownet2-pytorch.git
!mv /content/flownet2-pytorch /content/flownet2pytorch
os.chdir('./flownet2pytorch')
# install custom layers
!bash install.sh

Cloning into 'flownet2-pytorch'...
remote: Enumerating objects: 557, done.
remote: Total 557 (delta 0), reused 0 (delta 0), pack-reused 557
Receiving objects: 100% (557/557), 6.28 MiB | 21.80 MiB/s, done.
Resolving deltas: 100% (312/312), done.
running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector

### Add packages to IPython system path

In [ ]:
import os
os.sys.path.append('/root/.local/lib/python3.6/site-packages/resample2d_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append('/root/.local/lib/python3.6/site-packages/correlation_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append( '/root/.local/lib/python3.6/site-packages/channelnorm_cuda-0.0.0-py3.6-linux-x86_64.egg')

# Download files and Install Packages

In [ ]:
!pip install pypng
!pip install tensorboardx
!pip install  setproctitle colorama scipy==1.1.0

     |████████████████████████████████| 655kB 5.3MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp37-none-any.whl size=67163 sha256=c1c81872ef4feb2859d051cd50d0d4a22cbeced20165503478f230ddbfc68e24
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d4722c2ae9486691b1d49b922b9877922beeabb3
Successfully built pypng
     |████████████████████████████████| 122kB 6.2MB/s 
     |████████████████████████████████| 31.2MB 52.5MB/s 
ERROR: pymc3 3.11.2 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da',dest_path='./FlowNet2_checkpoint.pth.tar')

# Run the inference

### Uploading sample video. 

In [ ]:
mkdir_ifnotexists('./videos')
gdd.download_file_from_google_drive(file_id='1DHxE-PakEtAs7IrxUiO0VAIw8Jq5PKMC',dest_path='./videos/ship.mp4')
gdd.download_file_from_google_drive(file_id='1V9N5FenOdJyDWKj8DKhuem6iCmVbVzRW',dest_path='./videos/snowplanet.mp4')
##gdd.download_file_from_google_drive(file_id='1IW7qO9hDG8VJVnYuJI1ov1pmDkoq3vMC',dest_path='./videos/skyhouse.mp4')
gdd.download_file_from_google_drive(file_id='1detmA3HBFp21GrifsLZvN-M4pOL_aPYp',dest_path='./videos/cartooncoaster.mp4')
##gdd.download_file_from_google_drive(file_id='1cpbRTupK1OohYsp5qAu83Lkfzmhr9ACz',dest_path='./videos/glowingdance.mp4')


# Visualizing flo files

### Define show_flow() for visualization.
 Original Source https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py

In [ ]:
# Source:https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py
import matplotlib.pyplot as plt
import numpy as np

UNKNOWN_FLOW_THRESH = 1e7
def show_flow(filename):
    """
    visualize optical flow map using matplotlib
    :param filename: optical flow file
    :return: None
    """
    flow = read_flow(filename)
    img = flow_to_image(flow)
    plt.imshow(img)
    plt.show()

def read_flow(filename):
    """
    read optical flow from Middlebury .flo file
    :param filename: name of the flow file
    :return: optical flow data in matrix
    """
    f = open(filename, 'rb')
    magic = np.fromfile(f, np.float32, count=1)
    data2d = None

    if 202021.25 != magic:
        print ('Magic number incorrect. Invalid .flo file')
    else:
        w = int(np.fromfile(f, np.int32, count=1)[0])
        h = int(np.fromfile(f, np.int32, count=1)[0])
        #print("Reading %d x %d flo file" % (h, w))
        data2d = np.fromfile(f, np.float32, count=2 * w * h)
        # reshape data into 3D array (columns, rows, channels)
        data2d = np.resize(data2d, (h, w, 2))
    f.close()
    return data2d

def flow_to_image(flow):
    """
    Convert flow into middlebury color code image
    :param flow: optical flow map
    :return: optical flow image in middlebury color
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999.
    maxv = -999.
    minu = 999.
    minv = 999.

    idxUnknow = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    maxu = max(maxu, np.max(u))
    minu = min(minu, np.min(u))

    maxv = max(maxv, np.max(v))
    minv = min(minv, np.min(v))

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    #print( "max flow: %.4f\nflow range:\nu = %.3f .. %.3f\nv = %.3f .. %.3f" % (maxrad, minu,maxu, minv, maxv))

    u = u/(maxrad + np.finfo(float).eps)
    v = v/(maxrad + np.finfo(float).eps)

    img = compute_color(u, v)

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return np.uint8(img)


def compute_color(u, v):
    """
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = make_color_wheel()
    ncols = np.size(colorwheel, 0)

    rad = np.sqrt(u**2+v**2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a+1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols+1] = 1
    f = fk - k0

    for i in range(0, np.size(colorwheel,1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0-1] / 255
        col1 = tmp[k1-1] / 255
        col = (1-f) * col0 + f * col1

        idx = rad <= 1
        col[idx] = 1-rad[idx]*(1-col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

    return img


def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

## Optical flow calculation

### .flo file to npy array file

In [ ]:
import numpy as np
import pandas as pd 
import os

def flo2csv(videoName, lon, lat):

  flo_pth='/content/flownet2pytorch/output/inference/run.epoch-0-flow-field/'
  flos=[flo_pth + f for f in os.listdir(flo_pth)]

  flo_npy_pth='/content/gdrive/MyDrive/Video_Rest_Frames/'
  mkdir_ifnotexists(flo_npy_pth)
  flo_npy_pth='/content/gdrive/MyDrive/Video_Rest_Frames/opticalflow/'
  mkdir_ifnotexists(flo_npy_pth)
  flo_npy_pth=flo_npy_pth+videoName+'/'
  mkdir_ifnotexists(flo_npy_pth)
  flo_npy_pth=flo_npy_pth+'horizontal_'+str(lon)+'_vertical_'+str(lat)+'.csv'
  if os.path.exists(flo_npy_pth):
        return
  flos = sorted(flos)
  for i in range(len(flos)):
    npyfiles = np.array([read_flow(flos[i])])
    npyfiles = npyfiles[0]
    c = np.sqrt(np.sum(npyfiles*npyfiles, axis=2))
    value =  np.matrix.sum(np.matrix(c))/npyfiles.shape[0]/npyfiles.shape[1]
    if i == 0:
      average_flo =   [value]
    else:
      average_flo.append(value)
    os.remove(flos[i])
  DF = pd.DataFrame(average_flo) 
  DF.to_csv(flo_npy_pth)
    


### Calculate optical flow and disparity

In [ ]:
from google.colab import files
video_pth='/content/flownet2pytorch/videos/'
videos=[video_pth + v for v in os.listdir(video_pth)]
videonames = [os.path.splitext(os.path.basename(v))[0] for v in os.listdir(video_pth)]
print(videonames)

['snowplanet', 'cartooncoaster', 'ship']


In [ ]:
# Clean disparity, output, frames folder
import shutil
import time
import multiprocessing as mp



for i in videonames:
  print(i)
  os.chdir('/content/flownet2pytorch')
  if os.path.exists('/content/flownet2pytorch/frames'):
    shutil.rmtree(r'/content/flownet2pytorch/frames')
  if os.path.exists('/content/flownet2pytorch/output'):
    shutil.rmtree(r'/content/flownet2pytorch/output')
  for theta in range(-180, 180, 15):
    for phi in range(90, -105, -15):
      flo_npy_pth='/content/gdrive/MyDrive/Video_Rest_Frames/opticalflow/'
      flo_npy_pth=flo_npy_pth+i+'/'
      flo_npy_pth=flo_npy_pth+'horizontal_'+str(theta)+'_vertical_'+str(phi)+'.csv'
      if os.path.exists(flo_npy_pth):
        continue
      t= time.time()
      print(flo_npy_pth)
      perspectiveSotre(theta, phi, i)
      print('Convert: '+str(time.time()-t)+'seconds')
      mkdir_ifnotexists('./output')
      t = time.time()
      # Generate .flo files using FlowNet2
      !python main.py --inference --model FlowNet2 --save_flow --save ./output --inference_dataset ImagesFromFolder --inference_dataset_root ./frames --resume ./FlowNet2_checkpoint.pth.tar
      print('Flo: '+str(time.time()-t)+'seconds')
      npy_pth = flo2csv(i, theta, phi)
      print('CSV: '+str(time.time()-t)+'seconds')
  print('Finished '+i)

snowplanet
Finished snowplanet
cartooncoaster
Finished cartooncoaster
ship
Finished ship
